In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import string
import csv

2023-09-11 17:16:10.989582: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 17:16:10.990921: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 17:16:11.023939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 17:16:11.024672: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 17:16:11.585496: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
df = pd.read_csv('Shakespeare_data.csv', nrows = 50000)
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [3]:
import csv

corpus = []

with open('Shakespeare_data.csv') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)        # to pass first row,header
    for row in reader:
        corpus.append(row[5])

corpus = corpus[:50000]       
print(len(corpus))
print(corpus[:3])


50000
['ACT I', 'SCENE I. London. The palace.', 'Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others']


In [4]:
import string

def text_cleaner(text):
    text = "".join(car for car in text if car not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')
    return text

corpus = [text_cleaner(line) for line in corpus[:50000]]


In [5]:
# Tokenization is the process of splitting up a text into a list of individual words, or tokens.
# corpus is too big if you try with all data, you can see this message
corpus = corpus[:3000]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_words = len(word_index) + 1
total_words

3759

In [6]:
# create input sequences using list of tokens
input_sequences =[]

for sentence in corpus:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        

In [7]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and label
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
# Create one-hot encoding of the labels
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

In [8]:

# Build and compile the GRU model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(GRU(150))
model.add(Dense(total_words, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(predictors, label, epochs=5, verbose=1)


Epoch 1/5
628/628 [==============================] - 9s 13ms/step - loss: 6.7732 - accuracy: 0.0417
Epoch 2/5
628/628 [==============================] - 8s 13ms/step - loss: 6.1057 - accuracy: 0.0647
Epoch 3/5
628/628 [==============================] - 8s 13ms/step - loss: 5.6902 - accuracy: 0.0922
Epoch 4/5
628/628 [==============================] - 8s 13ms/step - loss: 5.3013 - accuracy: 0.1100
Epoch 5/5
628/628 [==============================] - 8s 13ms/step - loss: 4.8953 - accuracy: 0.1331


In [9]:
seed_text = "help me in this"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    c = np.argmax(predicted, axis = 1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == c:
            output_word = word
            break
    seed_text += " " + output_word
    if len(seed_text) % 10 == 0:
        seed_text += '\n'
print(seed_text)

help me in this same
 camp of shrewsbury
 and the day of the
 day


In [10]:
seed_text = "Love all, trust a few"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    c = np.argmax(predicted, axis = 1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == c:
            output_word = word
            break
    seed_text += " " + output_word
    if len(seed_text) % 10 == 0:
        seed_text += '\n'
print(seed_text)

Love all, trust a few flocks of the day of the day
 of the day


In [11]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.callbacks import EarlyStopping

def build_model(hp):
    model = Sequential()
    model.add(Embedding(total_words, hp.Int('embedding_dim', min_value=32, max_value=128, step=32), input_length=max_sequence_len-1))
    model.add(GRU(units=hp.Int('gru_units', min_value=32, max_value=128, step=32), dropout=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(total_words, activation=hp.Choice('dense_activation',values=['relu','sigmoid','softmax']),kernel_initializer='he_normal'))
    model.compile(optimizer='adam', loss=hp.Choice('loss_fn',values=['binary_crossentropy','categorical_crossentropy']), metrics=['accuracy'])
    return model

# Initialize Keras Tuner RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tuner_dir',
    project_name='lstm_sentiment'
)

# Perform hyperparameter search
tuner.search(predictors, label, validation_split=0.2, epochs=3)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps)

# Build the final model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(predictors, label, validation_split=0.2, epochs=5, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
model.summary()

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.03560756891965866

Best val_accuracy So Far: 0.09138446301221848
Total elapsed time: 00h 01m 28s
INFO:tensorflow:Oracle triggered exit
Best Hyperparameters:
Epoch 1/5
502/502 [==============================] - 7s 11ms/step - loss: 6.8150 - accuracy: 0.0406 - val_loss: 6.6352 - val_accuracy: 0.0632
Epoch 2/5
502/502 [==============================] - 5s 11ms/step - loss: 6.0683 - accuracy: 0.0832 - val_loss: 6.5018 - val_accuracy: 0.0876
Epoch 3/5
502/502 [==============================] - 6s 11ms/step - loss: 5.6277 - accuracy: 0.1179 - val_loss: 6.5354 - val_accuracy: 0.0951
Epoch 4/5
502/502 [==============================] - 5s 11ms/step - loss: 5.2273 - accuracy: 0.1444 - val_loss: 6.6057 - val_accuracy: 0.0941
Epoch 5/5
502/502 [==============================] - 6s 11ms/step - loss: 4.8444 - accuracy: 0.1727 - val_loss: 6.7230 - val_accuracy: 0.0986
Model: "sequential_1"
________________________________________________________________

In [13]:
# Print the chosen activation function and loss function
best_activation = best_hps.get('dense_activation')
best_loss_function = best_hps.get('loss_fn')
best_em_dim = best_hps.get('embedding_dim')
best_units = best_hps.get('gru_units')
print("Chosen embedding dimension:", best_em_dim)
print("Chosen number of GRU units:", best_units)
print("Chosen Activation Function:", best_activation)
print("Chosen Loss Function:", best_loss_function)

Chosen embedding dimension: 96
Chosen number of LSTM units: 96
Chosen Activation Function: softmax
Chosen Loss Function: categorical_crossentropy


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(GRU(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(predictors, label, epochs=5,  verbose=1)

In [ ]:
seed_text = "help me in this"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    c = np.argmax(predicted, axis = 1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == c:
            output_word = word
            break
    seed_text += " " + output_word
    if len(seed_text) % 10 == 0:
        seed_text += '\n'
print(seed_text)

In [ ]:
seed_text = "Love all, trust a few"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    c = np.argmax(predicted, axis = 1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == c:
            output_word = word
            break
    seed_text += " " + output_word
    if len(seed_text) % 10 == 0:
        seed_text += '\n'
print(seed_text)